In [9]:
import yfinance as yf

# Välj aktie (t.ex. Apple)
ticker = yf.Ticker("AXFO.ST")

# Hämta kvartalsvisa resultaträkningar
quarterly_financials = ticker.quarterly_financials
quarterly_balance_sheet = ticker.quarterly_balance_sheet
quarterly_cashflow = ticker.quarterly_cashflow



In [10]:
# Visa data
print(quarterly_financials.T)


            Tax Effect Of Unusual Items  Tax Rate For Calcs  \
2025-06-30                          0.0            0.216625   
2025-03-31                          0.0            0.217617   
2024-12-31                          0.0            0.284585   
2024-09-30                          0.0            0.216009   
2024-06-30                          0.0            0.216398   

            Normalized EBITDA  \
2025-06-30       1.961000e+09   
2025-03-31       1.735000e+09   
2024-12-31       1.587000e+09   
2024-09-30       1.844000e+09   
2024-06-30       1.654000e+09   

            Net Income From Continuing Operation Net Minority Interest  \
2025-06-30                                        614000000.0            
2025-03-31                                        451000000.0            
2024-12-31                                        349000000.0            
2024-09-30                                        704000000.0            
2024-06-30                                        57

In [11]:
print(quarterly_balance_sheet.T)

            Treasury Shares Number  Ordinary Shares Number  Share Issued  \
2025-06-30                921517.0             215921723.0   216843240.0   
2025-03-31               1098345.0             215744895.0   216843240.0   
2024-12-31               1098345.0             215744895.0   216843240.0   
2024-09-30               1098345.0             215744895.0   216843240.0   
2024-06-30                950345.0             215892895.0   216843240.0   
2024-03-31                     NaN                     NaN           NaN   
2023-12-31                     NaN                     NaN           NaN   

                Net Debt    Total Debt  Tangible Book Value  Invested Capital  \
2025-06-30  2.120000e+09  1.542400e+10        -1.022000e+09      8.719000e+09   
2025-03-31  2.926000e+09  1.584200e+10        -1.600000e+09      8.883000e+09   
2024-12-31  2.167000e+09  1.529600e+10        -1.550000e+08      9.955000e+09   
2024-09-30  8.550000e+08  1.073500e+10         1.539000e+09      7.

In [12]:
print(quarterly_cashflow.T)

            Free Cash Flow  Repurchase Of Capital Stock  Repayment Of Debt  \
2025-06-30    1.458000e+09                  -25000000.0      -2.379000e+09   
2025-03-31    8.610000e+08                          0.0      -2.089000e+09   
2024-12-31    1.390000e+09                          0.0      -1.356000e+09   
2024-09-30    6.700000e+08                  -38000000.0      -1.001000e+09   
2024-06-30    5.180000e+08                  -28000000.0      -3.590000e+08   
2024-03-31             NaN                          NaN                NaN   
2023-12-31             NaN                          NaN                NaN   

            Issuance Of Debt  Issuance Of Capital Stock  Capital Expenditure  \
2025-06-30      1.612000e+09                        NaN         -471000000.0   
2025-03-31      2.447000e+09                        NaN         -371000000.0   
2024-12-31      2.702000e+09                        NaN         -482000000.0   
2024-09-30      9.410000e+08                        NaN

# Test Temporal TTM Rankings with Updated Configuration

Now testing our enhanced temporal analysis with lowered requirements (3 quarters minimum) to work with Swedish stock data availability.

In [14]:
# Test temporal analysis with updated configuration
import pickle
from RatioCalculator import RatioCalculator
from FinancialDataProcessor import FinancialDataProcessor
import pandas as pd

# Load raw data
with open('data/local/raw_financial_data.pkl', 'rb') as f:
    raw_data = pickle.load(f)

# Test with AAK (same as your yfinance test)
ticker = 'AAK'
ticker_data = raw_data[ticker]

# Format data for FinancialDataProcessor
quarterly_data = ticker_data.get('quarterly', {})
formatted_ticker_data = {
    'balance_sheet': quarterly_data.get('balance_sheet'),
    'income_statement': quarterly_data.get('income_statement'),
    'cash_flow': quarterly_data.get('cash_flow'),
    'current_price': quarterly_data.get('current_price'),
    'shares_outstanding': quarterly_data.get('shares_outstanding'),
    'market_cap': quarterly_data.get('market_cap'),
    'dividendRate': quarterly_data.get('dividendRate'),
    'dividends': quarterly_data.get('dividends'),
    'latest_report_date': quarterly_data.get('latest_report_date'),
    'info': quarterly_data.get('info', {})
}

print(f"Testing temporal analysis for {ticker}")
print(f"Available quarters: {quarterly_data['balance_sheet'].shape[0] if 'balance_sheet' in quarterly_data else 'N/A'}")

Testing temporal analysis for AAK
Available quarters: 6


In [27]:
# Process data and test temporal analysis
processor = FinancialDataProcessor('ratios_config.yaml')
processed_data = processor._process_single_ticker(ticker, formatted_ticker_data)

# Check TTM time series
ttm_series = processed_data['quarterly_ttm_series']
print(f"TTM series available: {list(ttm_series.keys())[:5]}...")
print(f"Revenue TTM series: {ttm_series['Total Revenue TTM']}")
print(f"TTM data points: {len(ttm_series['Total Revenue TTM'])}")

# Convert to DataFrame format for RatioCalculator
processed_df = pd.DataFrame([processed_data])
processed_df.index = [ticker]

print(f"\nDataFrame shape: {processed_df.shape}")
print(f"Has quarterly_ttm_series: {'quarterly_ttm_series' in processed_df.columns}")

TypeError: 'module' object is not callable

In [26]:
# Create proper test data with all required fields for temporal analysis
import pandas as pd

# Recreate the processed data structure with all required fields
ticker = "AAK"

# Create test data with all the required fields that ratios expect
processed_data = {
    'ticker': ticker,
    # ROE required fields
    'Net Income TTM': 500_000_000,  # 500M SEK
    'Stockholders Equity': 2_723_000_000,  # 2.7B SEK (gives ROE ≈ 0.1836)
    
    # ROIC required fields  
    'EBIT TTM': 600_000_000,  # 600M SEK
    'Tax Provision TTM': 100_000_000,  # 100M SEK
    'Pretax Income TTM': 600_000_000,  # 600M SEK (gives tax rate ≈ 0.167)
    'Total Debt': 1_000_000_000,  # 1B SEK
    # Stockholders Equity already defined above
    
    # Profitability required fields
    'Gross Profit TTM': 1_400_000_000,  # 1.4B SEK
    'Total Revenue TTM': 5_020_000_000,  # 5B SEK (gives gross margin ≈ 0.279)
    'Operating Income TTM': 513_000_000,  # 513M SEK (gives operating margin ≈ 0.102)
    
    # Add the quarterly TTM time series for temporal analysis
    'quarterly_ttm_series': {
        # ROE time series (calculated from Net Income / Stockholders Equity over time)
        'Net Income TTM': [450_000_000, 475_000_000, 500_000_000],
        'Stockholders Equity': [2_600_000_000, 2_650_000_000, 2_723_000_000],
        
        # ROIC components time series
        'EBIT TTM': [520_000_000, 560_000_000, 600_000_000],
        'Tax Provision TTM': [87_000_000, 93_000_000, 100_000_000],
        'Pretax Income TTM': [520_000_000, 560_000_000, 600_000_000],
        'Total Debt': [1_100_000_000, 1_050_000_000, 1_000_000_000],
        
        # Profitability time series
        'Gross Profit TTM': [1_250_000_000, 1_325_000_000, 1_400_000_000],
        'Total Revenue TTM': [4_800_000_000, 4_900_000_000, 5_020_000_000],
        'Operating Income TTM': [400_000_000, 456_000_000, 513_000_000],
    }
}

# Create DataFrame
processed_df = pd.DataFrame([processed_data])

print("🔍 Debug temporal perspective extraction:")
print(f"Created fresh processed_df with {len(processed_df)} rows")
print(f"Data has quarterly_ttm_series: {'quarterly_ttm_series' in processed_df.iloc[0]}")

first_row = processed_df.iloc[0]
if 'quarterly_ttm_series' in first_row:
    ttm_series = first_row['quarterly_ttm_series']
    print(f"TTM series type: {type(ttm_series)}")
    if isinstance(ttm_series, dict):
        print(f"TTM series has {len(ttm_series)} time series")
        for key, values in list(ttm_series.items())[:3]:
            print(f"  {key}: {len(values) if isinstance(values, list) else 'not a list'} quarters")
    
# Check configuration
min_quarters = calculator.config['data_quality']['min_quarters_for_trend']
print(f"Min quarters for trend: {min_quarters}")

# Test perspective data extraction directly
from RatioCalculator import TemporalPerspective
test_perspective = TemporalPerspective(name='trend_ttm', weight=0.333, description='test', quarters_back=1)

print(f"\nTesting _extract_perspective_data with quarters_back=1:")
perspective_data = calculator._extract_perspective_data(processed_df, test_perspective.quarters_back)
print(f"✅ Perspective data available: {perspective_data is not None}")

# Test a single ratio calculation with the correct method name
result = calculator.calculate_ratios_from_processed_data(ticker, processed_df)
sample_ratios = ['ROE', 'ROIC', 'Bruttomarginal']

print("\n🎉 Updated Temporal Analysis Results:")
print(f"Overall score: {result.overall_score}")
print(f"Data quality score: {result.data_quality_score:.1%}")

temporal_working = True
for ratio_name in sample_ratios:
    ratio_result = result.ratio_results[ratio_name]
    print(f"\n{ratio_name}:")
    print(f"  Current TTM: {ratio_result.current_value}")
    print(f"  Trend TTM: {ratio_result.trend_value}")
    print(f"  Stability TTM: {ratio_result.stability_value}")
    print(f"  Composite Score: {ratio_result.composite_score}")
    if ratio_result.calculation_notes:
        print(f"  Notes: {ratio_result.calculation_notes[0]}")
    
    if ratio_result.trend_value is None or ratio_result.stability_value is None:
        temporal_working = False

print(f"\n✅ All 3 temporal perspectives working: {temporal_working}")

# Show which temporal perspectives are being used
print(f"\n📊 Temporal perspectives configured:")
for perspective in calculator.temporal_perspectives:
    print(f"  {perspective.name}: quarters_back={perspective.quarters_back}, weight={perspective.weight}")

2025-09-17 18:04:01,705 - INFO - Starting ratio calculation for AAK from processed data
2025-09-17 18:04:01,706 - WARNING - Low data quality for AAK: 61.7%
2025-09-17 18:04:01,722 - INFO - Completed ratio calculation for AAK - Overall score: 3.106
2025-09-17 18:04:01,706 - WARNING - Low data quality for AAK: 61.7%
2025-09-17 18:04:01,722 - INFO - Completed ratio calculation for AAK - Overall score: 3.106


🔍 Debug temporal perspective extraction:
Created fresh processed_df with 1 rows
Data has quarterly_ttm_series: True
TTM series type: <class 'dict'>
TTM series has 9 time series
  Net Income TTM: 3 quarters
  Stockholders Equity: 3 quarters
  EBIT TTM: 3 quarters
Min quarters for trend: 3

Testing _extract_perspective_data with quarters_back=1:
✅ Perspective data available: True

🎉 Updated Temporal Analysis Results:
Overall score: 3.1060586535048404
Data quality score: 61.7%

ROE:
  Current TTM: 0.1836210062431142
  Trend TTM: -2.9121572072416138
  Stability TTM: 10.0
  Composite Score: 2.4313974450674998

ROIC:
  Current TTM: 0.13430029546065
  Trend TTM: -6.649959662017476
  Stability TTM: 10.0
  Composite Score: 1.1702854309365769

Bruttomarginal:
  Current TTM: 0.2788844621513944
  Trend TTM: -3.367163577021097
  Stability TTM: 10.0
  Composite Score: 2.3116030547483897

✅ All 3 temporal perspectives working: True

📊 Temporal perspectives configured:
  current_ttm: quarters_back=0, 